In [1]:
from pyspark.sql import SparkSession

# Create SparkSession instance
spark = SparkSession.builder \
        .appName("GCS Files Load") \
        .getOrCreate()


24/05/12 12:32:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
#load form GCS
gcs_path = "gs://228bucket/train.csv"

# read CSV file
df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(gcs_path)

df.show()

+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2009-06-15 17:26:21|        4.5|2009-06-15 17:26:21|      -73.844311|      40.721319|        -73.84161|       40.712278|              1|
|2010-01-05 16:52:16|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|2011-08-18 00:35:00|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|2012-04-21 04:30:42|        7.7|2012-04-21 04:30:42|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|2010-03-09 07:51:00|        5.3|2

In [6]:
row_count = df.count()
column_count = len(df.columns)
print(f"DataFrame shape: {row_count} rows, {column_count} columns")

DataFrame shape: 55423856 rows, 8 columns


24/05/12 12:47:01 ERROR TransportClient: Failed to send RPC RPC 8764269613026743268 to /10.128.0.11:43312: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException: null
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source) ~[netty-transport-4.1.100.Final.jar:4.1.100.Final]
24/05/12 12:47:01 WARN BlockManagerMasterEndpoint: Error trying to remove broadcast 9 from block manager BlockManagerId(12, cluster-9ee9-w-0.us-central1-f.c.data228project-423109.internal, 43329, None)
java.io.IOException: Failed to send RPC RPC 8764269613026743268 to /10.128.0.11:43312: io.netty.channel.StacklessClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:392) ~[spark-network-common_2.12-3.5.0.jar:3.5.0]
	at org.apache.spark.network.client.TransportClient$StdChannelListener.operationComplete(TransportClient.java:369) ~[spark-network-common_2.12-3.5

In [7]:
# extreme values
from pyspark.sql import functions as F

columns = df.columns
expressions = {col: [F.max(col).alias(col + '_max'), F.min(col).alias(col + '_min')] for col in columns}
aggregated_df = df.agg(*[expr for sublist in expressions.values() for expr in sublist])
aggregated_df.show()

+-------------------+-------------------+---------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+---------------------+---------------------+--------------------+--------------------+-------------------+-------------------+
|            key_max|            key_min|fare_amount_max|fare_amount_min|pickup_datetime_max|pickup_datetime_min|pickup_longitude_max|pickup_longitude_min|pickup_latitude_max|pickup_latitude_min|dropoff_longitude_max|dropoff_longitude_min|dropoff_latitude_max|dropoff_latitude_min|passenger_count_max|passenger_count_min|
+-------------------+-------------------+---------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+---------------------+---------------------+--------------------+--------------------+-------------------+-------------------+
|2015-06-30 23:59:54|2009-01-01 00

In [9]:
summary_df = df.describe()
summary_df.show()

24/05/12 13:18:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|summary|       fare_amount|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|   passenger_count|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|  count|          55423856|          55423856|         55423856|          55423480|         55423480|          55423856|
|   mean|11.345045601663854|-72.50968444358728|39.91979178688818| -72.5112097297181|39.92068144482884|1.6853799201556816|
| stddev|  20.7108321982325| 12.84888338140265|9.642353041994935|12.782196517830771|9.633345796415126|1.3276643570959683|
|    min|            -300.0|      -3442.059565|     -3492.263768|      -3442.024565|     -3547.886698|                 0|
|    max|          93963.36|       3457.625683|      3408.789565|        3457.62235|      3537.132528|               208|
+-------+---------------

In [10]:
#count in years
from pyspark.sql import functions as F

df_with_year = df.withColumn("year", F.year(F.col("pickup_datetime")))
yearly_counts = df_with_year.groupBy("year").count()
yearly_counts = yearly_counts.orderBy("year")
yearly_counts.show()

+----+-------+
|year|  count|
+----+-------+
|2009|8543166|
|2010|8354997|
|2011|8841346|
|2012|8919666|
|2013|8655190|
|2014|8252682|
|2015|3856809|
+----+-------+



In [11]:
# Filter out data from 2010 to 2014
filtered_df = df_with_year.filter(F.col("year").between(2010, 2014))

# Group and count filtered data
yearly_counts = filtered_df.groupBy("year").count()
yearly_counts.orderBy("year").show()

+----+-------+
|year|  count|
+----+-------+
|2010|8354997|
|2011|8841346|
|2012|8919666|
|2013|8655190|
|2014|8252682|
+----+-------+



In [16]:
from pyspark.sql import functions as F

# Extract year from timestamp
df_with_year = df.withColumn("year", F.year(F.col("pickup_datetime")))

# Filter out data from 2010 to 2014
years_of_interest = [2010, 2011, 2012, 2013, 2014]
filtered_df = df_with_year.filter(F.col("year").isin(years_of_interest))

# Remove 'year' auxiliary column from results
final_df = filtered_df.drop("year")

final_df.show(5)

+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2010-01-05 16:52:16|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|2011-08-18 00:35:00|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|2012-04-21 04:30:42|        7.7|2012-04-21 04:30:42|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|2010-03-09 07:51:00|        5.3|2010-03-09 07:51:00|      -73.968095|      40.768008|       -73.956655|       40.783762|              1|
|2011-01-06 09:50:45|       12.1|2

In [17]:
final_df = final_df.drop("key")
final_df.show(2)

+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
only showing top 2 rows



In [18]:
final_df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|       fare_amount|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|   passenger_count|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|          43023881|          43023881|          43023881|          43023505|          43023505|          43023881|
|   mean| 11.45069147179111|-72.38663522466115|39.844684480190836|-72.38069175554979| 39.84129464692027|1.6849287259789512|
| stddev|13.704557609147619|13.801440280030436|10.631201080467019|13.741225332629819|10.626293480383582|1.3313331336418934|
|    min|           -112.56|      -3442.059565|      -3492.263768|      -3442.024565|      -3547.886698|                 0|
|    max|          61550.86|       3457.625683|       3408.789565|        3457.62235|       3537.132528|               208|
+-------

In [24]:
from pyspark.sql.functions import col
# Delete rows in the pickup_latitude&dropoff_latitude column that are not in the range -90 to 90
final_df_filtered = final_df.filter((col("pickup_latitude") >= -90) & (col("pickup_latitude") <= 90) & (col("dropoff_latitude") >= -90) & (col("dropoff_latitude") <= 90))

print("\nafter deletion DataFrame:")
final_df_filtered.describe().show()


after deletion DataFrame:


+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|summary|       fare_amount|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|   passenger_count|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|  count|          43022085|          43022085|         43022085|          43022085|         43022085|          43022085|
|   mean|11.450657893499478|-72.38521253502451|39.84203364306933| -72.3807124762978|39.84034224936179|1.6849488349995125|
| stddev|13.704575667893463|11.523495790997185|6.440936122161879|11.426139813424136|6.446743068257062|1.3313309664796518|
|    min|           -112.56|      -3050.559458|       -80.880182|      -3050.559458|       -80.880182|                 0|
|    max|          61550.86|       2228.738685|        89.742163|       2157.301527|        89.816665|               208|
+-------+---------------

In [25]:
# Delete rows in the pickup_longitude&dropoff_longitude column that are not in the range -180 to 180
final_df_filtered2 = final_df_filtered.filter((col("pickup_longitude") >= -180) & (col("pickup_longitude") <= 180) & (col("dropoff_longitude") >= -180) & (col("dropoff_longitude") <= 180))

print("\nafter deletion DataFrame:")
final_df_filtered2.describe().show()


after deletion DataFrame:


+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|summary|       fare_amount|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|   passenger_count|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|  count|          43020789|          43020789|         43020789|          43020789|         43020789|          43020789|
|   mean|11.450659353784879|-72.37262984888063|39.84209287433402|-72.37048475161347|39.84050055917293|1.6849332772581183|
| stddev| 13.70467975423845|10.904398190067518|6.440626341324525|10.908544200149183|6.446145832009294|1.3312916902410423|
|    min|           -112.56|       -168.603534|       -77.833874|        -173.95763|       -74.354612|                 0|
|    max|          61550.86|        169.972765|        89.742163|        169.972765|        89.816665|               208|
+-------+---------------

In [27]:
final_df_filtered3 = final_df_filtered2.filter((col("fare_amount") >= 0) & (col("fare_amount") <= 2000))

print("\nafter deletion DataFrame:")
final_df_filtered3.describe().show()


after deletion DataFrame:


+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+
|summary|      fare_amount|  pickup_longitude|  pickup_latitude| dropoff_longitude|  dropoff_latitude|   passenger_count|
+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+
|  count|         43019680|          43019680|         43019680|          43019680|          43019680|          43019680|
|   mean|11.44945041641536|-72.37267830493542|39.84211884743914|-72.37055213498593| 39.84053689331871|1.6849273402312617|
| stddev|9.880095848122915| 10.90424559085245|6.440558841611602|10.908330430495125|6.4460470707882935|1.3312888325130892|
|    min|              0.0|       -168.603534|       -77.833874|        -173.95763|        -74.354612|                 0|
|    max|           1564.5|        169.972765|        89.742163|        169.972765|         89.816665|               208|
+-------+---------------

In [31]:
final_df_filtered4 = final_df_filtered3.filter((col("passenger_count") > 0))

print("\nafter deletion DataFrame:")
final_df_filtered4.describe().show()


after deletion DataFrame:


+-------+-----------------+------------------+-----------------+------------------+-----------------+------------------+
|summary|      fare_amount|  pickup_longitude|  pickup_latitude| dropoff_longitude| dropoff_latitude|   passenger_count|
+-------+-----------------+------------------+-----------------+------------------+-----------------+------------------+
|  count|         42826493|          42826493|         42826493|          42826493|         42826493|          42826493|
|   mean|11.46060764560048|-72.37126634451855|39.84122739241672| -72.3694076343133|39.83978624861877|1.6925279172403866|
| stddev|9.891615592207698|10.909384908586414|6.445068729955861|10.912602527780848|6.450125250236515|1.3294587658656984|
|    min|              0.0|       -168.603534|       -77.833874|        -173.95763|       -74.354612|                 1|
|    max|           1564.5|        169.972765|        89.742163|        169.972765|        89.816665|               208|
+-------+-----------------+-----

In [32]:
#save file
output_path = 'gs://228bucket/processed_train_1.0.csv'

final_df_filtered4.write.mode('overwrite').option('header', 'true').csv(output_path)

In [33]:
#check saved file
#load form GCS
new_path = "gs://228bucket/processed_train_combined.csv"

# read CSV file
df_new = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(new_path)

df_new.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|summary|       fare_amount|  pickup_longitude|  pickup_latitude|dropoff_longitude|  dropoff_latitude|   passenger_count|
+-------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|  count|          42826493|          42826493|         42826493|         42826493|          42826493|          42826493|
|   mean|11.460607645597449|-72.37126634451735|39.84122739241672|-72.3694076343127|39.839786248618445|1.6925279172403866|
| stddev| 9.891615592207685|10.909384908586341|6.445068729955884|10.91260252778086| 6.450125250236519|1.3294587658656971|
|    min|               0.0|       -168.603534|       -77.833874|       -173.95763|        -74.354612|                 1|
|    max|            1564.5|        169.972765|        89.742163|       169.972765|         89.816665|               208|
+-------+---------------